# Databases

In [4]:
import sqlite3

## Create DB

- Create a new sqlite3 database called `library`.

In [14]:
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

## Create Table

Create a table named `books` with the following characteristics:
- id INTEGER PRIMARY KEY
- title TEXT
- author TEXT
- published_date DATE

In [15]:
cursor.execute('''CREATE TABLE books
                  (id INTEGER PRIMARY KEY,
                  title TEXT,
                  author TEXT,
                  published_date DATE)''')
conn.commit()

## Add Book

Write a Python function `add_book()` that will add a new book to the "books" table.

In [17]:
def add_book(conn, cursor, title, author, published_date):
    cursor.execute('''INSERT INTO books (title, author, published_date)
                      VALUES (?, ?, ?)''', (title, author, published_date))
    conn.commit()


In [18]:
add_book(conn, cursor, 'The Lord of the Rings', 'J.R.R. Tolkien', '1954-07-29')
add_book(conn, cursor, 'Harry Potter', 'J.K. Rowling', '1997-06-26')

## Get Book

Write a Python function `get_books_by_author()` that will retrieve all books by a given author.

In [24]:
def get_books_by_author(conn, cursor, author):
    cursor.execute('SELECT * FROM books WHERE author = ?', (author,))
    conn.commit()
    return cursor.fetchall()

In [25]:
get_books_by_author(conn, cursor, 'J.R.R. Tolkien')

[(1,
  'Lord of the Rings: The Fellowship of the Ring',
  'J.R.R. Tolkien',
  '1954-07-29')]

## Update Book

Write a Python function `update_book_title()` that will update the title of a book given its id.

In [26]:
def update_book_title(conn, cursor, book_id, new_title):
    cursor.execute('UPDATE books SET title = ? WHERE id = ?', (new_title, book_id))
    conn.commit()

In [27]:
update_book_title(conn, cursor, 1, 'Lord of the Rings: The Fellowship of the Ring')

In [28]:
get_books_by_author(conn, cursor, 'J.R.R. Tolkien')

[(1,
  'Lord of the Rings: The Fellowship of the Ring',
  'J.R.R. Tolkien',
  '1954-07-29')]

## Delete Book

Write a Python function `delete_book()` that will delete a book from the "books" table given its id.

In [29]:
def delete_book(conn, cursor, book_id):
    cursor.execute('DELETE FROM books WHERE id = ?', (book_id,))
    conn.commit()

In [30]:
delete_book(conn, cursor, 2)

In [31]:
get_books_by_author(conn, cursor, 'J.K. Rowling')

[]

## Published Before

Write a Python function `get_books_published_before()` that will retrieve all books published before a given date.

In [32]:
def get_books_published_before(conn, cursor, date):
    cursor.execute('SELECT * FROM books WHERE published_date < ?', (date,))
    conn.commit()
    return cursor.fetchall()

In [33]:
get_books_published_before(conn, cursor, '2000-01-01')

[(1,
  'Lord of the Rings: The Fellowship of the Ring',
  'J.R.R. Tolkien',
  '1954-07-29')]

## Closing

Close the cursor and the connection.

In [34]:
cursor.close()
conn.close()

## Guess the Movie

Create a game where the user is presented with characters from a movie title replaced by underscores and must guess the movie title. Users have limited attempts, and their scores are recorded in a database.

### Create DB

Create a new database called "guess_movie.db".

In [36]:
conn = sqlite3.connect('guess_movie.db')
cursor = conn.cursor()

### Tables Setup

Create a table called "movies" with columns "id" and "title" and another table "scores" with columns "id," "username," and "score."

In [37]:
cursor.execute('''CREATE TABLE IF NOT EXISTS movies 
                  (id INTEGER PRIMARY KEY, title TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS scores (
                  id INTEGER PRIMARY KEY,
                  username TEXT,
                  score INTEGER)''')

conn.commit()

### Populate DB

Initialize the DB with the following movies:

In [38]:
movie_titles = [
    "The Shawshank Redemption",
    "The Godfather",
    "The Dark Knight",
    "Pulp Fiction",
    "Forrest Gump",
    "Inception",
    "Fight Club",
    "The Matrix",
    "The Social Network",
    "Jurassic Park",
    "Titanic",
    "Toy Story",
    "Finding Nemo",
    "The Lord of the Rings: The Return of the King",
    "Star Wars: Episode IV - A New Hope",
    "The Silence of the Lambs",
    "Se7en",
    "The Lion King",
    "Back to the Future",
    "The Usual Suspects"
]

In [39]:
for title in movie_titles:
    cursor.execute('INSERT INTO movies (title) VALUES (?)', (title,))

conn.commit()

### The Game

- You should create a Tkinter window with a label to display the masked movie title, an entry widget to input the guessed movie title, a button to submit the guess, and a label to display the remaining attempts.
- When the game starts, pick a random movie title from the database, mask it with underscores, and display it to the user. Each time the user submits a guess, check it against the actual movie title. If the guess is correct, the user wins, and their score is updated in the database.

A good way to organize the code may be:
- Create a `MovieDatabase` class that encapsulates all functionalities related to database management, including inserting movie titles, fetching a random movie, and closing the database connection.
- Create a `GuessTheMovieGame` class that handles all the game logic and GUI functionalities, and it utilizes an instance of `MovieDatabase` to interact with the database.
- Define a `main()` function that runs the game, and ensures that the database connection is closed appropriately upon application termination.

This is just a proposal, feel free to organize your program as you wish.

In [44]:
import random

class MovieDatabase:
    def __init__(self, db_name):
        self.connection = sqlite3.connect(db_name)
        self.cursor = self.connection.cursor()

    def insert_movie(self, title):
        self.cursor.execute("INSERT INTO movies (title) VALUES (?)", (title,))
        self.connection.commit()

    def fetch_random_movie(self):
        self.cursor.execute("SELECT * FROM movies")
        self.connection.commit()
        movies = self.cursor.fetchall()
        return random.choice(movies)

    def close_connection(self):
        self.connection.close()

In [45]:
import tkinter as tk

class GuessTheMovieGame:
    def __init__(self, movie_db):
        self.movie_db = movie_db
        self.movie_title = self.fetch_movie_title()
        self.display_title = self._get_display_title()
        self.attempts_left = 3
        self.setup_gui()

    def fetch_movie_title(self):
        movie = self.movie_db.fetch_random_movie()
        return movie[1]

    def check_guess(self):
        guess = self.entry.get()
        if guess.lower() == self.movie_title.lower():
            self.label.config(text="Congratulations! Correct Guess!")
        else:
            self.attempts_left -= 1
            if self.attempts_left == 0:
                self.label.config(text=f"Sorry! The correct movie was: {self.movie_title}")
            else:
                self.label.config(text=f"Try Again! Attempts left: {self.attempts_left}")

    def setup_gui(self):
        self.root = tk.Tk()
        self.root.title("Guess the Movie")
        
        self.label = tk.Label(self.root, text=f"Guess the movie title: {self.display_title}")
        self.label.pack(pady=10)

        self.entry = tk.Entry(self.root)
        self.entry.pack(pady=10)

        self.button = tk.Button(self.root, text="Check", command=self.check_guess)
        self.button.pack(pady=10)

    def run(self):
        self.root.mainloop()

    def _get_display_title(self):
        # Randomly replace 30% of the letters with underscores
        n_underscores = 0.3 * len(self.movie_title)
        rand_idxs = random.sample(range(len(self.movie_title)), int(n_underscores))
        display_title = list(self.movie_title) # Convert to list of characters
        for i in rand_idxs:
            display_title[i] = '_'
        return ''.join(display_title)

In [46]:
def main():
    movie_db = MovieDatabase('guess_movie.db')

    # Setup game and run.
    game = GuessTheMovieGame(movie_db)
    game.run()

    # Close database connection.
    movie_db.close_connection()

In [47]:
main()

### Improvements

- Create a "High Scores" section that lists the top 10 scores from the "scores" table.
- Include a hint feature, revealing a character of the movie title after a certain number of incorrect guesses.
- Add an admin section where you can add/delete movies to/from the database.
- Add multiplayer functionality.